In [29]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy import stats


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [30]:
data_f = pd.read_csv("/home/akki/Downloads/datasets_89775_207354_play_tennis.csv")


In [31]:
data_f.head()

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [32]:
data_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   day       14 non-null     object
 1   outlook   14 non-null     object
 2   temp      14 non-null     object
 3   humidity  14 non-null     object
 4   wind      14 non-null     object
 5   play      14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


In [33]:
class Tree(object):
    "Generic tree node."
    def __init__(self, name='root', children=None):
        self.name = name
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)
    def __repr__(self):
        return self.name
    
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)
    

In [34]:
class Tree(object):
    "Generic tree node."
    def __init__(self, name='root', children=None):
        self.name = name
        self.children = []
        if children is not None:
            for i in children:
                self.add_child(i)
    def __repr__(self):
        return self.name
    
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)
    

In [35]:
def Entropy(labels):
    entropy = 0
    labels = list(labels)
    categories = np.unique(labels)
    n = len(labels)
    for j in categories:
        p = labels.count(j)/n
        entropy = entropy + (-p)*(np.log2(p))
    return entropy

In [36]:
def Information_Gain(S,labels,data_f):
    S = list(S)
    labels = list(labels)
    n = len(S)
    entropy_S = Entropy(labels)
    F = np.unique(S)
    entropy_Sv = []
    for v in F:
        l = []
        for i in range(n):
            if S[i]==v:
                l.append(labels[i])
        entropy_Sv.append(Entropy(l))
    information_gain = entropy_S
    for j in range(len(F)):
        num_of_v = S.count(F[j])
        information_gain = information_gain - (num_of_v/n)*(entropy_Sv[j])
    return information_gain
    

In [37]:
def Imp_attr(data_f,attr,labels):
    info_gain = []
    for A in attr:
        gain = Information_Gain(data_f[A],labels,data_f)
        info_gain.append(gain)
    max_gain = info_gain.index(max(info_gain))
    return attr[max_gain]

In [40]:
def decision_tree(data_f,attr,labels_name):
    if len(np.unique(list(data_f[labels_name]))) == 1:
        C = np.unique(list(data_f[labels_name]))[0]
        return Tree(C)
    if len(attr) == 0:
        C = stats.mode(data_f[labels_name])[0][0]
        return Tree(C)
    else:
        imp_attr = Imp_attr(data_f,attr,data_f[labels_name])
        tree = Tree(imp_attr)
        attr.remove(imp_attr)
        for a in np.unique(data_f[imp_attr]):
            new_data_f = data_f[data_f[imp_attr]==a]
            if (len(new_data_f) == 0 ):
                C = stats.mode(data_f[labels_name])[0][0]
                tree.add_child(Tree(C))
            else:
                tree.add_child(decision_tree(new_data_f,attr,labels_name))
    return tree

In [41]:
attr = ['outlook','temp','humidity','wind']
tree = decision_tree(data_f,attr,labels_name='play')

In [42]:
def print_tree(tree):
    if len(tree.children)!=0:
        print("parent node: " ,tree)
        print("childern nodes :")
        for child in tree.children:
            print("\t\t", child)
    
    for child in tree.children:
        if len(tree.children)!=0:
            print_tree(child)


In [8]:
def print_tree(tree):
    if len(tree.children)!=0:
        print("parent node: " ,tree)
        print("childern nodes :")
        for child in tree.children:
            print("\t\t", child)
    
    for child in tree.children:
        if len(tree.children)!=0:
            print_tree(child)


In [43]:
print_tree(tree)

parent node:  outlook
childern nodes :
		 Yes
		 wind
		 humidity
parent node:  wind
childern nodes :
		 No
		 Yes
parent node:  humidity
childern nodes :
		 No
		 Yes
